In [ ]:
import anndata
import urllib.request

# Load data
url = "https://drive.google.com/uc?export=download&id=1aZpy_l9xDQm5s0g17ztuizAFNbp5PX4L"
path, _ = urllib.request.urlretrieve(url)
adata = anndata.read_h5ad(path)
adata

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

# Preprocess data
log_counts = sc.pp.log1p(adata, copy=True)
sc.pp.highly_variable_genes(log_counts, n_top_genes=200)
adata = adata[:, log_counts.var['highly_variable']]

# Extract B cells and regulatory T cells
selected_cells = adata.obs['phenoid'].isin(['b.cells', 'regulatory.t'])
adata = adata[selected_cells]
adata.obs['cell_type'] = adata.obs['phenoid'].astype('category')

In [ ]:
from scdesigner.simulator import scdesigner
from scdesigner.margins.marginal import NB

sim = scdesigner(adata, NB("~ cell_type"), max_epochs=10)

In [ ]:
sim.parameters()

In [ ]:
print(sim)

In [ ]:
sim.nullify("cell_type", gene_list)